### **Problem 4: Store-Level Stock Insights and Cleanup**

Management wants to detect patterns in store sales and clean up inventory tags:

* Parse `Product Code` to extract product family (`PLA`, `TPU`, etc.).
* Verify that barcodes follow a 13-digit numeric standard.
* Remove or flag invalid barcodes.
* Group by store city and compute average total per transaction and most sold material.
* For each store, compute median unit price for PLA materials only.
* Extract numerical store building numbers and convert them to integers.

*Hint: You may use string length and isdigit() checks for barcode validation.*


In [205]:
import pandas as pd
import numpy as np
import re

In [206]:
data = pd.read_csv('fila_heat_filament_sales_april2025.csv')

In [207]:
df = pd.DataFrame(data)

In [208]:
df.head(3)

,Date Purchased,Receipt Number,Customer Name,Customer Address,Phone Number,Email,Store Location,Product Name,Product Code,Bar Code,Material Name,Color,Weight,Supplier,Lot Number,Price,Quantity,Tax,Total Price
0,2025-04-01,1ff49b78-8946-4e85-b59c-de66bacfb3d0,Danielle Johnson,"3321 Brittany Bypass, North Jefferyhaven, 79408",8386379402,danielle.johnson@hotmail.com,"5423 Garcia Light, West Melanieview, 06196",Standard PLA Filament,PLA-792,6.184960e+12,PLA,Blue,500,3DFilaments,L5012,26.69,1,1.87,28.56
1,2025-04-01,434308bc-89fa-4a68-8fb5-d27bbeb79919,Tracie Wyatt,"64752 Kelly Skyway, Jacquelineland, 80341",+1-283-276-4835x0305,tracie.wyatt@yahoo.com,"1395 Diana Locks, Thomasberg, 32826",Flexible TPU Filament,TPU-338,9.696530e+12,TPU,Purple,500,ProtoPolymers,L1520,20.88,2,2.92,44.68
2,2025-04-01,52fbe43b-9954-4eb4-8025-7ad1eb2263dd,Eric Moore,"691 James Mountain, Tashatown, 89667",001-184-514-6270x4828,eric.moore@gmail.com,"489 Eric Track, New Stephanie, 70015",Flexible TPU Filament,TPU-325,7.015430e+12,TPU,Purple,1000,PrintPro,L4257,41.47,4,11.61,177.49


In [209]:
# 1.0 Parse `Product Code` to extract product family (`PLA`, `TPU`, etc.).
product_family_pattern = r'^([A-Z]+)'

In [210]:
df['Product Family'] = df['Product Code'].str.extract(product_family_pattern)

In [211]:
df[['Product Code', 'Product Family']].head(3)

,Product Code,Product Family
0,PLA-792,PLA
1,TPU-338,TPU
2,TPU-325,TPU


In [212]:
# 2. * Verify that barcodes follow a 13-digit numeric standard.
df['Bar Code'] = df['Bar Code'].astype(str)

In [213]:
valid_barcode_pattern = r'^\d{13}$'

In [214]:
df['Valid Barcode'] = df['Bar Code'].str.match(valid_barcode_pattern)

In [215]:
invalid_barcodes = df[~df['Valid Barcode']]

In [216]:
invalid_barcodes.head(3)

,Date Purchased,Receipt Number,Customer Name,Customer Address,Phone Number,Email,Store Location,Product Name,Product Code,Bar Code,...,Color,Weight,Supplier,Lot Number,Price,Quantity,Tax,Total Price,Product Family,Valid Barcode
0,2025-04-01,1ff49b78-8946-4e85-b59c-de66bacfb3d0,Danielle Johnson,"3321 Brittany Bypass, North Jefferyhaven, 79408",8386379402,danielle.johnson@hotmail.com,"5423 Garcia Light, West Melanieview, 06196",Standard PLA Filament,PLA-792,6184960000000.0,...,Blue,500,3DFilaments,L5012,26.69,1,1.87,28.56,PLA,False
1,2025-04-01,434308bc-89fa-4a68-8fb5-d27bbeb79919,Tracie Wyatt,"64752 Kelly Skyway, Jacquelineland, 80341",+1-283-276-4835x0305,tracie.wyatt@yahoo.com,"1395 Diana Locks, Thomasberg, 32826",Flexible TPU Filament,TPU-338,9696530000000.0,...,Purple,500,ProtoPolymers,L1520,20.88,2,2.92,44.68,TPU,False
2,2025-04-01,52fbe43b-9954-4eb4-8025-7ad1eb2263dd,Eric Moore,"691 James Mountain, Tashatown, 89667",001-184-514-6270x4828,eric.moore@gmail.com,"489 Eric Track, New Stephanie, 70015",Flexible TPU Filament,TPU-325,7015430000000.0,...,Purple,1000,PrintPro,L4257,41.47,4,11.61,177.49,TPU,False


In [217]:
# 3. Remove or flag invalid barcodes.
df['Invalid Barcode Flag'] = df['Valid Barcode'].apply(lambda x: '' if x else 'FLAGGED') 

In [218]:
df[['Material Name', 'Lot Number', 'Bar Code', 'Invalid Barcode Flag']].head(3)

,Material Name,Lot Number,Bar Code,Invalid Barcode Flag
0,PLA,L5012,6184960000000.0,FLAGGED
1,TPU,L1520,9696530000000.0,FLAGGED
2,TPU,L4257,7015430000000.0,FLAGGED


In [219]:
# Alternative -> more pythonic
df['Invalid Barcode Flag'] = ~df['Valid Barcode']

In [220]:
df['Invalid Barcode Flag'] = df['Invalid Barcode Flag'].map({True: 'FLAGGED', False: ''})

In [221]:
df[['Material Name', 'Lot Number', 'Bar Code', 'Invalid Barcode Flag']].head(3)

,Material Name,Lot Number,Bar Code,Invalid Barcode Flag
0,PLA,L5012,6184960000000.0,FLAGGED
1,TPU,L1520,9696530000000.0,FLAGGED
2,TPU,L4257,7015430000000.0,FLAGGED


In [222]:
# 4. Group by store city and compute average total per transaction and most sold material.
df['Store City'] = df['Store Location'].str.split(', ', expand=True)[1]

In [223]:
# Average total price per transaction per city
avg_total_by_city = df.groupby('Store City')['Total Price'].mean().reset_index()

In [224]:
avg_total_by_city

,Store City,Total Price
0,Alvarezburgh,53.91
1,Alyssafurt,180.66
2,Amberland,105.40
3,Ambermouth,93.67
4,Andersonport,125.23
...,...,...
351,Wilsonton,148.62
352,Woodmouth,149.97
353,Wrightberg,45.60
354,Wrightland,130.58


In [225]:
# Most sold material per City
most_sold = (
    df.groupby(['Store City', 'Material Name'])['Total Price']
    .sum()
    .reset_index()
    .sort_values(['Store City', 'Total Price'], ascending=[True, False])
    .drop_duplicates(subset='Store City')
)

In [226]:
print(type(most_sold))

<class 'pandas.core.frame.DataFrame'>


In [227]:
most_sold

,Store City,Material Name,Total Price
0,Alvarezburgh,PLA,53.91
1,Alyssafurt,PLA,180.66
2,Amberland,PLA,105.40
3,Ambermouth,ABS,93.67
4,Andersonport,PLA,125.23
...,...,...,...
354,Wilsonton,TPU,148.62
355,Woodmouth,PLA,149.97
356,Wrightberg,PLA,45.60
357,Wrightland,TPU,130.58


In [228]:
# Optional: Combine both
summary = avg_total_by_city.merge(most_sold, on='Store City')
summary.rename(columns={'Total Price_x': 'Avg Transaction', 'Total Price_y': 'Top Material Sales'}, inplace=True)


In [229]:
# 5. For each store, compute median unit price for PLA materials only.
# filter PLA
pla_only = df[df['Material Name'] == 'PLA'].copy()

In [230]:
# compute unit price
pla_only['Unit Price'] = pla_only['Total Price'] / pla_only['Weight']

In [231]:
median_unit_price_per_store = pla_only.groupby('Store City')['Unit Price'].median().reset_index()

In [232]:
median_unit_price_per_store.rename(columns={'Unit Price': 'Median PLA Unit Price'}, inplace=True)

In [233]:
median_unit_price_per_store.head(3)

,Store City,Median PLA Unit Price
0,Alvarezburgh,0.107820
1,Alyssafurt,0.240880
2,Amberland,0.140533


In [234]:
# 6. Extract numerical store building numbers and convert them to integers.
building_num_pattern = r'^([0-9]+)'

In [235]:
df['Building Number'] = df['Store Location'].str.extract(building_num_pattern)

In [236]:
df['Building Number'] = df['Building Number'].astype(int)

In [237]:
df[['Building Number', 'Store Location']].head(3)

,Building Number,Store Location
0,5423,"5423 Garcia Light, West Melanieview, 06196"
1,1395,"1395 Diana Locks, Thomasberg, 32826"
2,489,"489 Eric Track, New Stephanie, 70015"
